<a href="https://colab.research.google.com/github/minit88/ML-DL/blob/main/Lab_05/logistic_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# For reproducibility
torch.manual_seed(1)

In [4]:
x_data = [[1,2],[2,3],[3,1],[4,3],[5,3],[6,2]] # 6 x 2
y_data = [[0],[0],[0],[1],[1],[1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [5]:
print(x_train.shape)
print(y_train.shape)

torch.Size([6, 2])
torch.Size([6, 1])


In [6]:
 print('e^1 equals:',torch.exp(torch.FloatTensor([1])))

e^1 equals: tensor([2.7183])


In [7]:
W=torch.zeros((2,1),requires_grad=True)
b= torch.zeros(1,requires_grad=True)

# **Hypothesis**

In [10]:
hypothesis = 1/(1+torch.exp(-(x_train.matmul(W)+b)))

In [12]:
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)
torch.Size([6, 1])


# **Hypothesis(Sigmoid)**

In [15]:
print('1/(1+e^{-1} equals:' ,torch.sigmoid(torch.FloatTensor([1])))

1/(1+e^{-1} equals: tensor([0.7311])


In [18]:
hypothesis = torch.sigmoid(x_train.matmul(W)+b)

In [19]:
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward0>)
torch.Size([6, 1])


# **Cost Function**

In [20]:
losses = -(y_train *torch.log(hypothesis)+(1-y_train)*(torch.log(1-hypothesis)))
cost = torch.mean(losses)

In [21]:
print('cost equals:',cost)

cost equals: tensor(0.6931, grad_fn=<MeanBackward0>)


# **앞의 과정을 축소화**

In [22]:
F.binary_cross_entropy(hypothesis,y_train)

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward0>)

# **Whole Training Procedure**

In [24]:
# 모델 초기화
W = torch.zeros((2,1),requires_grad=True)
b=torch.zeros(1,requires_grad=True)
#optimizer 설정
optimizer = optim.SGD([W,b],lr =1) # learning W and b with lr

nb_epochs =1000
for epoch in range(nb_epochs+1):
  
  #cost
  hypothesis = torch.sigmoid(x_train.matmul(W)+b)
  cost = F.binary_cross_entropy(hypothesis,y_train)

  optimizer.zero_grad() # reset grad
  cost.backward() 
  optimizer.step()
 
  if epoch % 100==0:
    print('Epoch {:4d}/{} cost:{:.6f}'.format(epoch,nb_epochs,cost.item()))


Epoch    0/1000 cost:0.693147
Epoch  100/1000 cost:0.134722
Epoch  200/1000 cost:0.080643
Epoch  300/1000 cost:0.057900
Epoch  400/1000 cost:0.045300
Epoch  500/1000 cost:0.037261
Epoch  600/1000 cost:0.031672
Epoch  700/1000 cost:0.027556
Epoch  800/1000 cost:0.024394
Epoch  900/1000 cost:0.021888
Epoch 1000/1000 cost:0.019852


# **Evaluation**

In [26]:
hypothesis = torch.sigmoid(x_train.matmul(W)+b)
print(hypothesis[:5])

tensor([[2.7648e-04],
        [3.1608e-02],
        [3.8977e-02],
        [9.5622e-01],
        [9.9823e-01]], grad_fn=<SliceBackward0>)


In [31]:
prediction = hypothesis>=torch.FloatTensor([0.5]) # if hypothesis higher than 0.5 -> 1
for i in prediction[:5]:
  if i==True:
    print(torch.FloatTensor([1]))
  else:
    print(torch.FloatTensor([0]))
print(y_train[:5])

tensor([0.])
tensor([0.])
tensor([0.])
tensor([1.])
tensor([1.])
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.]])


In [32]:
correct_prediction = prediction.float() == y_train
print(correct_prediction[:5])

tensor([[True],
        [True],
        [True],
        [True],
        [True]])


In [52]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.linear(x))

In [54]:
model = BinaryClassifier()

In [55]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 100
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = model(x_train)

    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    if epoch % 10 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format(
            epoch, nb_epochs, cost.item(), accuracy * 100,
        ))

Epoch    0/100 Cost: 0.733120 Accuracy 50.00%
Epoch   10/100 Cost: 0.471225 Accuracy 83.33%
Epoch   20/100 Cost: 0.739847 Accuracy 66.67%
Epoch   30/100 Cost: 0.542283 Accuracy 83.33%
Epoch   40/100 Cost: 0.451816 Accuracy 83.33%
Epoch   50/100 Cost: 0.367630 Accuracy 83.33%
Epoch   60/100 Cost: 0.288506 Accuracy 83.33%
Epoch   70/100 Cost: 0.220184 Accuracy 83.33%
Epoch   80/100 Cost: 0.173718 Accuracy 100.00%
Epoch   90/100 Cost: 0.151886 Accuracy 100.00%
Epoch  100/100 Cost: 0.140431 Accuracy 100.00%
